In [10]:
#%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import solve as linsolve

## Select Options for $\Gamma$ Calculation

In [2]:
def parZ(z1, z2):
    return 1./(1./z1 + 1./z2)

def gamma(zl, zo):
    return (zl - zo)/(zl + zo)

In [3]:
R1 = 25 # 75
R2 = 50
C = 10e-12
L = 100e-12
zo = 50

f = np.linspace(1, 10000, 10000)*1e6
jw = 1j*2*np.pi*f

In [4]:
rr = parZ(R1, R2)
Grr = gamma(rr, zo) + 0*f

rc = parZ(R1, 1./(jw*C))
Grc = gamma(rc, zo)

lc = parZ(1./(jw*C), jw*L)
Glc = gamma(lc, zo)

rlc = parZ(parZ(R1, 1./(jw*C)), jw*L)
Grlc = gamma(rlc, zo)


In [5]:
fig, axs = plt.subplots(2)

xfcn = lambda x_ : x_/1e9
yfcn = lambda y_ : np.abs(y_)
yfcn2 = lambda y_ : np.unwrap(np.angle(y_))

axs[0].plot(xfcn(f), yfcn(Grr), label='R Parallel')
axs[1].plot(xfcn(f), yfcn2(Grr), label='R Parallel')

axs[0].plot(xfcn(f), yfcn(Grc), label='RC Parallel')
axs[1].plot(xfcn(f), yfcn2(Grc), label='RC Parallel')

axs[0].plot(xfcn(f), yfcn(Glc), label='LC Parallel')
axs[1].plot(xfcn(f), yfcn2(Glc), label='LC Parallel')

axs[0].plot(xfcn(f), yfcn(Grlc), label='RLC Parallel')
axs[1].plot(xfcn(f), yfcn2(Grlc), label='RLC Parallel')

axs[1].legend(loc='best')
axs[0].set_ylabel('Magnitude')
axs[1].set_ylabel('Phase (rad)')
axs[1].set_xlabel('Frequency (GHz)')
fig.suptitle('Reflection Coefficients for Different Loads')

axs[0].grid(True)
axs[1].grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Voltage Calculation for the Line

### RC and RL Terminated Lines

In [6]:
def zin(zl, zo, f, l, vp):
    w = 2*np.pi*f
    beta = w/vp
    
    zin = zo*(zl + 1j*zo*np.tan(beta*l))/(zo + 1j*zl*np.tan(beta*l))
    return zin

def vplus(zo, zs, zl, f, l, vp, Vs):
    jw = 2j*np.pi*f
    jbeta = jw/vp
    
    gamma_ = gamma(zl, zo)
    zin_ = zin(zl, zo, f, l, vp)
    
    vplus = (zin_ / (zin_ + zs)) * Vs / (np.e**(jbeta*l) + gamma_*np.e**(-jbeta*l))
    return vplus

def v_spectrum_at_z(z, zo, zs, zl, f, l, vp, Vs):
    # V(z) = Vplus (e^-jbz + G e^jbz)
    jw = 2j*np.pi*f
    jbeta = jw/vp
    vplus_ = vplus(zo, zs, zl, f, l, vp, Vs)
    v_ = vplus_ * (np.e**(-jbeta*z) + gamma(zl,zo)*np.e**(jbeta*z))
    return v_

def v_line_at_f(f0, zo, zs, zl, z_, l, vp, Vs):
    jw = 2j*np.pi*f0
    jbeta = jw/vp
    vplus_ = vplus(zo, zs, zl, f0, l, vp, Vs)
    v_ = vplus_ * (np.e**(-jbeta*z_) + gamma(zl,zo)*np.e**(jbeta*z_))
    return v_
    

In [7]:
Vs = 1
Zs = 50
R = 100
C = 100e-12
L = 10e-9
zo = 50
vp = 2e10 # cm/s
l = 5 # cm

z_ = np.linspace(-l, 0, 1001)
f = np.linspace(1, 10000, 10000)*1e6
jw = 1j*2*np.pi*f

jbeta = jw/vp

In [8]:
# Plot the spectrum at different points along the line.
fig, axs = plt.subplots(2)

xfcn = lambda x_ : x_/1e9
yfcn = lambda y_ : np.abs(y_)
yfcn2 = lambda y_ : np.unwrap(np.angle(y_))
#yfcn2 = lambda y_ : (np.angle(y_))

Zload = parZ(R, 1./(jw*C))

for z0 in [-l, -3*l/4, -l/2, -l/4, 0]:
    v_ = v_spectrum_at_z(z0, zo, Zs, Zload, f, l, vp, Vs)
    axs[0].plot(xfcn(f), yfcn(v_), label='z = %f' % z0)
    axs[1].plot(xfcn(f), yfcn2(v_), label='z = %f' % z0)

axs[1].legend(loc='best')
axs[0].set_ylabel('Magnitude')
axs[1].set_ylabel('Phase (rad)')
axs[1].set_xlabel('Frequency (GHz)')
fig.suptitle('Voltage at z w.r.t. Frequency')

axs[0].grid(True)
axs[1].grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
# Plot the spectrum at different points along the line.
fig, axs = plt.subplots(2)

xfcn = lambda x_ : x_
yfcn = lambda y_ : np.abs(y_)
yfcn2 = lambda y_ : np.unwrap(np.angle(y_))
#yfcn2 = lambda y_ : (np.angle(y_))

for f0 in [1, 1e8, 2e8, 5e9, 10e9]: # [1, 2.5e9, 5e9, 7.5e9, 10e9]:
    Zload = parZ(R, 1./(2j*np.pi*f0*C))
    v_ = v_line_at_f(f0, zo, Zs, Zload, z_, l, vp, Vs)
    axs[0].plot(xfcn(z_), yfcn(v_), label='f = %f' % (f0/1e9))
    axs[1].plot(xfcn(z_), yfcn2(v_), label='f = %f' % (f0/1e9))

axs[1].legend(loc='best')
axs[0].set_ylabel('Magnitude')
axs[1].set_ylabel('Phase (rad)')
axs[1].set_xlabel('z (cm)')
fig.suptitle('Voltage at Frequency w.r.t. z')

axs[0].grid(True)
axs[1].grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …